In [ ]:
%pip install python-dotenv azure-ai-ml docker

### Imports and .env variables

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

subscription_id = os.getenv("SUBSCRIPTION_ID")
resource_group_name = "insurance_claim"
workspace_name = "fastapi_ml_app"
tenant_id = os.getenv("TENANT_ID")

### Azure Login and Client

In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
ml_client = MLClient(
    credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group_name,
    workspace_name=workspace_name
    )



## Managed Endpoints

### Register the model

In [ ]:
from azure.ai.ml.entities import Model

model = Model(
    path="./endpoint_deployment/trained_models/Random_Forest.pkl",  # path to model dir or file
    name="insurance_model",
    description="Random Forest model of the first task at g7cr",
    type="custom_model",
    version=1
)

registered_model = ml_client.models.create_or_update(model)


### Set up the environment

In [ ]:
from azure.ai.ml.entities import Environment

env = Environment(
    name="insurance-endpt-env",
    description="Custom env for Random Forest scoring",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04",  # base image
    conda_file="./endpoint_deployment/environment.yml",          # or requirements_file
)

ml_client.environments.create_or_update(env)


Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'insurance-endpt-env', 'description': 'Custom env for Random Forest scoring', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/92e84d68-fcd6-4b50-af37-65156d8d5358/resourceGroups/insurance_claim/providers/Microsoft.MachineLearningServices/workspaces/fastapi_ml_app/environments/insurance-endpt-env/versions/1', 'Resource__source_path': '', 'base_path': 'c:\\Pyth\\G7-CR\\AzureML', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x000001A87FDD81D0>, 'serialize': <msrest.serialization.Serializer object at 0x000001A8002F1D90>, 'version': '1', 'conda_file': {'channels': ['defaults', 'conda-forge'], 'dependencies': ['python=3.13.2', 'pip', 'scikit-learn=1.6.1'

### Create and Deploy the Managed Endpoint

In [ ]:
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    CodeConfiguration
)
from azure.ai.ml.entities import ManagedOnlineEndpoint
import uuid

endpoint_name = f"insurance-endpt-{uuid.uuid4().hex[:4]}"

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Managed endpoint for my model",
    auth_mode="key"
)

deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=registered_model.id,  # registered model ID
    code_configuration=CodeConfiguration(
        code="./endpoint_deployment",       # folder with score.py and dependencies
        scoring_script="score.py"
    ),
    environment=env,
    instance_type="Standard_DS1_v2",
    instance_count=1
)

endpoint_poller = ml_client.online_endpoints.begin_create_or_update(endpoint)
endpoint_result = endpoint_poller.result()  # Waits for completion
deployment_poller = ml_client.online_deployments.begin_create_or_update(deployment)
deployment_result = deployment_poller.result()

Instance type Standard_DS1_v2 may be too small for compute resources. Minimum recommended compute SKU is Standard_DS3_v2 for general purpose endpoints. Learn more about SKUs here: https://learn.microsoft.com/azure/machine-learning/referencemanaged-online-endpoints-vm-sku-list
Check: endpoint insurance-endpt-24ca exists


............................................................................

### Update the traffic

In [ ]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
scoring_uri = endpoint.scoring_uri

endpoint.traffic = {"blue": 100}
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

Readonly attribute principal_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>
Readonly attribute tenant_id will be ignored in class <class 'azure.ai.ml._restclient.v2022_05_01.models._models_py3.ManagedServiceIdentity'>


In [ ]:
# endpoint_name = "insurance-endpt-24ca"
# endpoint = ml_client.online_endpoints.get(endpoint_name)
# print(f"Endpoint scoring URI: {endpoint.scoring_uri}")

Endpoint scoring URI: https://insurance-endpt-24ca.eastus2.inference.ml.azure.com/score


### Invoke the managed endpoint

In [ ]:
ml_client.online_endpoints.invoke(
    endpoint_name=endpoint_name,
    request_file="./endpoint_deployment/sample-request.json"
)


'{"predictions": ["Accepted", "Denied", "Denied", "Accepted"]}'

## AKS

### Build and Push Image

In [ ]:
import docker
import os

# DockerHub credentials
DOCKER_USERNAME = 'pramoda04'
DOCKER_PASSWORD = 'pramoda@04'
IMAGE_NAME = 'insuranceclaim'
TAG = 'latest'
FULL_IMAGE_NAME = f'{DOCKER_USERNAME}/{IMAGE_NAME}:{TAG}'

# Path to the directory containing your Dockerfile
dockerfile_path = r'C:\Pyth\G7-CR\Insurance_claim'

def build_and_push_image():
    client = docker.from_env()

    # Step 1: Build the Docker image
    print(f'Building image {FULL_IMAGE_NAME}...')
    image, build_logs = client.images.build(path=dockerfile_path, tag=FULL_IMAGE_NAME)
    for log in build_logs:
        if 'stream' in log:
            print(log['stream'].strip())

    # Step 2: Log in to Docker Hub
    print('Logging in to Docker Hub...')
    client.login(username=DOCKER_USERNAME, password=DOCKER_PASSWORD)

    # Step 3: Push the Docker image
    print(f'Pushing image {FULL_IMAGE_NAME}...')
    push_logs = client.images.push(repository=FULL_IMAGE_NAME, stream=True, decode=True)
    for log in push_logs:
        if 'status' in log:
            print(log['status'])

    print('Image pushed successfully!')


build_and_push_image()


### Setup AKS and create cluster

In [10]:
import os
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.containerservice import ContainerServiceClient
from kubernetes import client as k8s_client, config as k8s_config, utils as k8s_utils

LOCATION = "eastus"
CLUSTER_NAME = "AutoAKSCluster"

# Get Resource Group
resource_client = ResourceManagementClient(credential, subscription_id)
resource_client.resource_groups.get(resource_group_name)

# Create AKS Cluster with system-assigned identity
aks_client = ContainerServiceClient(credential, subscription_id)

print("Creating AKS cluster...")
aks_cluster_poller = aks_client.managed_clusters.begin_create_or_update(
    resource_group_name,
    CLUSTER_NAME,
    {
        "location": LOCATION,
        "dns_prefix": "fastapidns",
        "agent_pool_profiles": [
            {
                "name": "nodepool1",
                "count": 1,
                "vm_size": "Standard_D2ls_v6",
                "os_type": "Linux",
                "type": "VirtualMachineScaleSets",
                "mode": "System"
            }
        ],
        "identity": {"type": "SystemAssigned"},
        "enable_rbac": True,
        "network_profile": {
            "network_plugin": "azure"
        }
    }
).result()
print("AKS cluster created.")



Creating AKS cluster...
AKS cluster created.


### Controls for deployed cluster

In [ ]:
# ---- Stop the AKS Cluster ----
def stop_cluster():
    print("Stopping AKS cluster...")
    aks_client.managed_clusters.begin_stop(
        resource_group_name=resource_group_name,
        resource_name=CLUSTER_NAME
    ).result()
    print("Cluster stopped.")

# ---- Start the AKS Cluster ----
def start_cluster():
    print("Starting AKS cluster...")
    aks_client.managed_clusters.begin_start(
        resource_group_name=resource_group_name,
        resource_name=CLUSTER_NAME
    ).result()
    print("Cluster started.")

# Call the one you want:
# stop_cluster()
start_cluster()



Stopping AKS cluster...
Cluster stopped.


### Fetch deployed cluster and apply the yaml

In [ ]:
# Get credentials for kubectl
print("Fetching AKS credentials...")
os.system(f"az aks get-credentials --resource-group {resource_group_name} --name {CLUSTER_NAME} --overwrite-existing")

# Deploy to AKS via Kubernetes
k8s_config.load_kube_config()

print("Deploying to AKS...")
k8s_utils.create_from_yaml(k8s_client.ApiClient(), "deployment.yaml")

print("Deployment initiated. It may take a few minutes for the LoadBalancer IP to be assigned.")


In [ ]:
os.system("kubectl get services")

### To apply changes to existing deployment

In [ ]:
os.system("kubectl apply -f deployment.yaml")